In [1]:
from keras.models import load_model, Model
from keras.layers import Input, Dense, Conv1D, Activation, Add, Multiply
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

/home/sudeep/.conda/envs/jazz_music/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Preprocessing
'''
def mu_law_encode(audio, quantization_channels):
        with K.name_scope('encode'):    
        mu = 1.00*(quantization_channels - 1)
            # Perform mu-law companding transformation (ITU-T, 1988).
            # Minimum operation is here to deal with rare large amplitudes caused
            # by resampling.
        safe_audio_abs = K.minimum(K.abs(audio), 1.0)
        magnitude = K.log(1 + mu * safe_audio_abs) / K.log(1 + mu)
        signal = K.sign(audio) * magnitude
        # Quantize signal to the specified number of levels.
        return K.round((signal + 1) / 2 * mu + 0.5)
'''
"""
def mu_law_decode(output, quantization_channels):
    '''Recovers waveform from quantized values.'''
    with K.name_scope('decode'):
        mu = quantization_channels - 1
        # Map values back to [-1, 1].
        signal = 2 * (tf.to_float(output) / mu) - 1
        # Perform inverse of mu-law transformation.
        magnitude = (1 / mu) * ((1 + mu)**abs(signal) - 1)
    return tf.sign(signal) * magnitude
"""

def encode(audio, quantization_channels):
    mu = 1.00*(quantization_channels - 1)
    safe_audio_abs = np.minimum(np.absolute(audio), 1.0)
    magnitude = np.log(1 + mu * safe_audio_abs) / np.log(1 + mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    return np.floor((signal + 1) / 2 * mu + 0.5)

def normalize(sample):
    rel1 = sample - np.min(sample, axis=-1)
    rel2 = np.max(sample, axis=-1) - np.min(sample, axis =-1)
    return 2*rel1/(rel2+1e-7) - 1    
    
def preprocessing(raw_audio, quantization_channels=256):
    audio = normalize(raw_audio)
    audio = encode(audio, quantization_channels)
    return audio

In [1]:
'''
import os
import numpy as np
from scipy.io import wavfile
def load_dataset():
    X = []
    path = "Piano_10_16kHz/"
    for x in os.listdir(path):
        _, w = wavfile.read(path + x)
        if w.shape[0] < 160124:
            w = np.concatenate([w, np.zeros((160124 - w.shape[0], 2))])
        assert(w.shape[0] == 160124)
        audio_left = preprocessing(w.T[0].astype(np.float32))
        audio_right = preprocessing(w.T[1].astype(np.float32))
        X.append(audio_left)
        X.append(audio_right)
    return np.array(X)
X = load_dataset()
X.shape
'''

'\nimport os\nimport numpy as np\nfrom scipy.io import wavfile\ndef load_dataset():\n    X = []\n    path = "Piano_10_16kHz/"\n    for x in os.listdir(path):\n        _, w = wavfile.read(path + x)\n        if w.shape[0] < 160124:\n            w = np.concatenate([w, np.zeros((160124 - w.shape[0], 2))])\n        assert(w.shape[0] == 160124)\n        audio_left = preprocessing(w.T[0].astype(np.float32))\n        audio_right = preprocessing(w.T[1].astype(np.float32))\n        X.append(audio_left)\n        X.append(audio_right)\n    return np.array(X)\nX = load_dataset()\nX.shape\n'

In [2]:
#np.save('Piano_half_hour_16kHz_10s_samples', X)
import numpy as np
X = np.load('Piano_half_hour_16kHz_10s_samples.npy')
print(X.shape)
X[0]

(408, 160124)


array([44., 44., 45., ..., 46., 49., 56.], dtype=float32)

In [3]:
np.random.shuffle(X)
X[0]

array([ 40.,  32.,  30., ..., 102.,  66.,  52.], dtype=float32)

In [18]:
X_train = X[:370]
X_validation = X[370:]
print(X_train.shape, X_validation.shape)
print(Y_train.shape)

((370, 160124), (38, 160124))
(370, 160123)


In [4]:
dilation_channels = 64
causal_channels = 32
dilation_kernel = 2
def residual_block(X, dilation_rate):
    F = Conv1D(dilation_channels, dilation_kernel, padding='causal', dilation_rate=dilation_rate)(X)
    G = Conv1D(dilation_channels, dilation_kernel, padding='causal', dilation_rate=dilation_rate)(X)
    F = Activation('tanh')(F)
    G = Activation('sigmoid')(G)
    Y = Multiply()([F, G])
    return Y

In [11]:
causal_kernel = 2
causal_channels = 32
dilation_rate = 2
quantization_channels = 256
def model(input_shape):
    X_input = Input(input_shape)
    X = Conv1D(causal_channels, causal_kernel, padding='causal', dilation_rate=1)(X_input)
    Y = residual_block(X, 1)
    S0 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 2)
    S1 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 4)
    S2 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 8)
    S3 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 16)
    S4 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 32)
    S5 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 64)
    S6 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 128)
    S7 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 256)
    S8 = Conv1D(dilation_channels, 1, padding="same")(Y)
    Y = Conv1D(causal_channels, 1, padding="same")(Y)
    X = Add()([X, Y])
    Y = residual_block(X, 512)
    S9 = Conv1D(dilation_channels, 1, padding="same")(Y)
    S = Add()([S0, S1, S2, S3, S4, S5, S6, S7, S8, S9])
    X = Activation('relu')(S)
    X = Conv1D(128, 1, padding="same")(X)
    X = Activation('relu')(X)
    X = Conv1D(256, 1, padding="same")(X)
    X = Activation('softmax')(X)
    return Model(inputs = X_input, outputs = X)

In [12]:
net = model((160124, 256))
print(net.input, net.output)

(<tf.Tensor 'input_3:0' shape=(?, 160124, 256) dtype=float32>, <tf.Tensor 'activation_23/truediv:0' shape=(?, 160124, 256) dtype=float32>)


In [13]:
net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 160124, 256)  0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 160124, 32)   16416       input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 160124, 64)   4160        conv1d_3[0][0]                   
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 160124, 64)   4160        conv1d_3[0][0]                   
__________________________________________________________________________________________________
activation

In [49]:
net.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [30]:
X_ = X[:2]
Y_ = np.concatenate([X[:2, 1:], X[:2, -1].reshape(X_.shape[0], 1)], axis = -1)
Y_

array([[55., 54., 52., ..., 61., 56., 56.],
       [24., 29., 37., ..., 33., 35., 35.]], dtype=float32)

In [39]:
from numpy import argmax
from keras.utils import to_categorical
# define example

encoded = to_categorical(X_)
encoded.shape

(2, 160124, 256)

In [37]:
onehot_initialization(X_)

IndexError: arrays used as indices must be of integer (or boolean) type

In [31]:
batch_size = 2
X_ = X[:2]
Y_ = np.concatenate([X[:2, 1:], X[:2, -1].reshape(X_.shape[0], 1)], axis = -1)

net.fit(x = X_, y = Y_, epochs = 1, batch_size = 2)

NameError: name 'net' is not defined